In [1]:
# DATS 6103 – Individual Project 1 - Max Grossman
    # Description: Describe top 15 countries' military spending as compared between 
    #             countries and other indicators of interest (specifically gdp & population)
    # Data Source: World Bank DataBank and SIPRI via World Bank DataBank

In [231]:
### Dependencies ###
import pandas as pd
import matplotlib.pyplot as plt
import os
from ipykernel import kernelapp as app
### Set to correct working directory ###

# if data folder not in current directory, ask user to provide
if("/mili_data" in [x[0].split('militaries')[1] for x in os.walk(os.getcwd())]):
    pass
else:
    neededDirectory = input("Provide directory with data: ")
    os.chdir(neededDirectory)

In [233]:
### Step 1: Clean and Organize the datasets ###

# The code below completes the following steps
    # 1. read in data
    # 2. group in list
    # 3. remove uneeded columns and rename year columns
    # 4. subset dataframes in list to only top 15 countries in terms of military spending 
    
# 1. read in data from /mili_data folder
militarySpending = pd.read_csv("mili_data/militarySpending.csv",delimiter='\t')
population = pd.read_csv ("mili_data/population.csv")
gdpUSD = pd.read_csv('mili_data/gdp_usd.csv')

In [234]:
# 2. add dataframes to list
miliDataList = [militarySpending,population,gdpUSD]

In [235]:
# 3. clean dataframes

#use for loop to remove specific column names in population and militarySpending
for i in range(0,len(miliDataList)):
    
    #set index to Country Codes
    miliDataList[i] = miliDataList[i].set_index('Country Name')
    miliDataList[i].index.name = None
    miliDataList[i] = miliDataList[i].sort_index(axis=1)
    
    #convert militarySpending to float, also set militarySpending ix names equal to those in population and gdpUSD
    if(i==0):
        
        miliDataList[i] = miliDataList[i].rename(index = {
            "USA":"United States",
            "China, P.R.":"China",
            "USSR/Russia":"Russian Federation",
            "UK": "United Kingdom",
            "Korea, South":"Korea, Rep."
        })
        
    if(i>0):
    
        #generate seriesCode, a list of column names that include the word series as well as "Country Code"
        seriesCode = [s for s in list(miliDataList[i]) if "Series".lower() in s.lower()]
        seriesCode.extend([s for s in list(miliDataList[i]) if "Country".lower() in s.lower()])

        #drop columns in seriesCode list
        miliDataList[i] = miliDataList[i].drop(seriesCode,axis=1)
    
        #generate newColumns, list with 'Country Name' and column names w/o [YR####] text provided in original dataset 
        newColumns = [s.split(" [")[0] for s in list(miliDataList[i]) if "YR".lower() in s.lower()]
    
        #set current data frame columns to those in newColumns
        miliDataList[i].columns = newColumns
            
    #covert all string place holders and NaNs to zero
    miliDataList[i] = miliDataList[i].replace(". .",0)
    miliDataList[i] = miliDataList[i].replace("..",0)
    miliDataList[i] = miliDataList[i].replace("NaN",0)
    miliDataList[i] = miliDataList[i].fillna(0)
    miliDataList[i] = miliDataList[i].replace("xxx",0)
    miliDataList[i] = miliDataList[i].replace("",0)
      
    #make sure all dataframes have values of type float
    miliDataList[i] = miliDataList[i].astype(float)

In [236]:
# 4. Subset dataframes in list to include only top 15 military spending countries in year 2015

#sort militarySpending by top 2015 spenders, subset first 15, grab index for subset, and covert subset index to a list
top15mili = miliDataList[0].sort_values('2015',ascending=False)[0:15].index.values.tolist()

for i in range(0,len(miliDataList)):

    miliDataList[i] = miliDataList[i].loc[top15mili]
    print(miliDataList[i])

                        1988      1989      1990      1991      1992  \
United States       293093.0  304085.0  306170.0  280292.0  305141.0   
China                    0.0   11403.0   10085.0    9954.0   12420.0   
Saudi Arabia         13355.0   12750.0   16355.0   16355.0   15360.0   
Russian Federation  246011.0  218760.0  219114.0       0.0       0.0   
United Kingdom       34304.0   33499.0   38944.0   42074.0   40775.0   
India                11346.0   10590.0   10537.0    8622.0    8083.0   
France               36105.0   35317.0   42590.0   42703.0   45123.0   
Japan                28216.0   27966.0   28800.0   32785.0   35999.0   
Germany              35097.0   33604.0   42319.0   39516.0   41966.0   
Korea, Rep.           7732.0    9469.0   10111.0   10957.0   11615.0   
Brazil                5874.0    8761.0    9236.0    6695.0    4994.0   
Italy                19621.0   19927.0   23376.0   24336.0   25002.0   
Australia             5836.0    6300.0    6704.0    7024.0    68

In [197]:
### Step 2: Generate desired indicators ###
